In [ ]:
import os
import numpy as np
import cv2
# from pycocotools.coco import COCO
# from pycocotools import mask as maskUtils
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
import random
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from segmentation_models_pytorch import Unet

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
def generate_mask(annotation_file, image_dir, mask_dir): 
    os.makedirs(mask_dir, exist_ok=True)
    # Initialize COCO API
    coco = COCO(annotation_file)

# Get all image IDs
    image_ids = coco.getImgIds()

# Define category mapping
    categories = coco.loadCats(coco.getCatIds())
    category_mapping = {cat['id']: cat['name'] for cat in categories}
    num_classes = len(category_mapping)
    print(f"Number of classes: {num_classes}")

    for image_id in tqdm(image_ids, desc="Generating Multi-Class Masks"):
    # Load image info
        image_info = coco.loadImgs(image_id)[0]
        height, width = image_info['height'], image_info['width']

    # Initialize blank multi-class mask
        multi_class_mask = np.zeros((height, width), dtype=np.uint8)

    # Get all annotations for the image
        ann_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(ann_ids)

        for ann in annotations:
            category_id = ann['category_id']
            ann_id = ann['id']

        # Generate mask for the annotation
            if ann['iscrowd']:
                rle = ann['segmentation']
                mask = maskUtils.decode(rle)
            else:
                mask = coco.annToMask(ann)

        # Assign category_id to mask pixels
        # Handle overlapping by assigning the category_id (latest object overwrites)
            multi_class_mask[mask > 0] = category_id

    # Save the multi-class mask as PNG
        image_filename = image_info['file_name']
        mask_filename = os.path.splitext(image_filename)[0] + '_mask.png'
        mask_path = os.path.join(mask_dir, mask_filename)
        cv2.imwrite(mask_path, multi_class_mask)

In [ ]:
# Directories for your datasets
train_annotation_file = 'wildfire-week8/train/_annotations.coco.json'
train_images_dir = 'wildfire-week8/train/original'
train_masks_dir = 'wildfire-week8/train/mask'

valid_annotation_file = 'wildfire-week8/valid/_annotations.coco.json'
valid_images_dir = 'wildfire-week8/valid/original'
valid_masks_dir = 'wildfire-week8/valid/mask'

test_annotation_file = 'wildfire-week8/test/_annotations.coco.json'
test_images_dir = 'wildfire-week8/test/original'
test_masks_dir = 'wildfire-week8/test/mask'

In [ ]:
generate_mask(train_annotation_file, train_images_dir, train_masks_dir)

In [ ]:
generate_mask(valid_annotation_file, valid_images_dir, valid_masks_dir)

In [ ]:
generate_mask(test_annotation_file, test_images_dir, test_masks_dir)

## Image Segmentation - Water

In [ ]:

# Initialize the model
unet_model_water = Unet(
    encoder_name="efficientnet-b3",  # Choose your backbone (e.g., "resnet34", "efficientnet-b3", etc.)
    encoder_weights="imagenet",  # Use pre-trained weights
    in_channels=3,  # Input channels (e.g., 3 for RGB images)
    classes=1  # Number of output classes
)



In [ ]:

print(unet_model_water )

In [ ]:
import torch
import torch.nn as nn

# Define a plain U-Net model
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        # Encoder
        self.enc1 = self.conv_block(3, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        
        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)
        
        # Decoder
        self.dec4 = self.conv_block(1024 + 512, 512)
        self.dec3 = self.conv_block(512 + 256, 256)
        self.dec2 = self.conv_block(256 + 128, 128)
        self.dec1 = self.conv_block(128 + 64, 64)
        
        # Final layer
        self.final = nn.Conv2d(64, 1, kernel_size=1)
    
    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),  # Add padding
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),  # Add padding
            nn.ReLU(inplace=True),
        )
    
    def forward(self, x):
        # Encoding
        enc1 = self.enc1(x)
        enc2 = self.enc2(nn.MaxPool2d(2)(enc1))
        enc3 = self.enc3(nn.MaxPool2d(2)(enc2))
        enc4 = self.enc4(nn.MaxPool2d(2)(enc3))
        
        # Bottleneck
        bottleneck = self.bottleneck(nn.MaxPool2d(2)(enc4))
        
        # Decoding
        dec4 = self.dec4(torch.cat((nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)(bottleneck), enc4), dim=1))
        dec3 = self.dec3(torch.cat((nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)(dec4), enc3), dim=1))
        dec2 = self.dec2(torch.cat((nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)(dec3), enc2), dim=1))
        dec1 = self.dec1(torch.cat((nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)(dec2), enc1), dim=1))
        
        # Final layer
        return self.final(dec1)

# Initialize the model
plain_unet = UNet()

# Test the model
input_tensor = torch.randn(1, 3, 640, 640)  # Batch size of 1, 3 channels (RGB), 640x640 image
output = plain_unet(input_tensor)
print(output.shape)


In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])
input_tensor = torch.randn(1, 3, 640, 640)  # Batch size of 1, 3 channels (RGB), 640x640 image
output = plain_unet(input_tensor)
print(output.shape)

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.image_filenames = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.masks_dir, self.image_filenames[idx].replace('.jpg', '')+'_mask.png')  # Adjust file extension if needed

        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Debugging prints
        if image is None:
            raise ValueError(f"Image not found or unable to load: {image_path}")
        if mask is None:
            raise ValueError(f"Mask not found or unable to load: {mask_path}")

        # Ensure proper dimensions
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError(f"Unexpected image shape: {image.shape}")

        # Resize or pad image and mask to be divisible by 32
        # image = self.resize_or_pad(image)
        # mask = self.resize_or_pad(mask)

        # Normalize image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # mask = mask // 51  # Adjust if necessary
        mask = np.where(mask == 4, 1, 0).astype('float32')
        if self.transform:
            image = self.transform(image)

        return torch.tensor(image, dtype=torch.float32).permute(0, 1, 2), torch.tensor(mask, dtype=torch.float32)



In [ ]:
import json
from collections import defaultdict
import os
from PIL import Image
import numpy as np

def count_classes_in_coco(coco_json_file, image_dir):
    """
    Counts the occurrences of each class (category) in the COCO dataset annotations, including the background.
    
    Args:
        coco_json_file (str): Path to the COCO format JSON annotation file.
        image_dir (str): Path to the directory containing the original images (to calculate background pixels).
        
    Returns:
        dict: A dictionary with class names as keys and their respective counts (in pixels) as values.
    """
    # Load the COCO JSON annotation file
    with open(coco_json_file, 'r') as f:
        coco_data = json.load(f)
    
    # Create a dictionary to store class counts
    class_counts = defaultdict(int)
    
    # Category mapping (id -> category name)
    category_mapping = {category['id']: category['name'] for category in coco_data['categories']}
    
    # To track pixels covered by objects
    covered_pixels_by_image = defaultdict(int)

    # Loop through all annotations
    for annotation in coco_data['annotations']:
        # Get the class id from the annotation (category_id)
        class_id = annotation['category_id']
        
        # Get the image id
        image_id = annotation['image_id']
        
        # Get the area of the segmented mask (to count the number of pixels)
        class_counts[class_id] += annotation['area']  # 'area' represents the number of pixels for the object
        
        # Accumulate the number of pixels covered by objects in each image
        covered_pixels_by_image[image_id] += annotation['area']
    
    # Include background calculation by analyzing total pixels in the images
    for image_info in coco_data['images']:
        image_id = image_info['id']
        image_path = os.path.join(image_dir, image_info['file_name'])
        
        # Open the image and calculate the total number of pixels
        image = Image.open(image_path)
        total_pixels = image.width * image.height
        
        # Calculate background pixels
        background_pixels = total_pixels - covered_pixels_by_image[image_id]
        
        # Add background class count
        class_counts[0] += background_pixels  # Assuming class 0 represents background

        
        keys = list(class_counts.keys())
        keys.sort()
        sorted_dict = {i: class_counts[i] for i in keys}
    return sorted_dict

# Example usage:
coco_json_path = "wildfire-week8/train/_annotations.coco.json"
image_directory = "wildfire-week8/train/original"
class_counts = count_classes_in_coco(coco_json_path, image_directory)
print(class_counts)


In [ ]:
total_background = class_counts[0] + class_counts[1] + class_counts[2] + class_counts[3]
total_foreground = class_counts[4]
pos_weight = total_background / total_foreground

print(pos_weight)

In [ ]:
# Create datasets
train_dataset_water = SegmentationDataset(train_images_dir, train_masks_dir, transform)
valid_dataset_water = SegmentationDataset(valid_images_dir, valid_masks_dir, transform)
test_dataset_water = SegmentationDataset(test_images_dir, test_masks_dir, transform)

# Create data loaders
train_loader_water = DataLoader(train_dataset_water, batch_size=4, shuffle=True)
valid_loader_water = DataLoader(valid_dataset_water, batch_size=4, shuffle=False)
test_loader_water = DataLoader(test_dataset_water, batch_size=4, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm


# Initialize the criterion with pos_weight
criterion = nn.BCEWithLogitsLoss(torch.tensor(pos_weight, dtype=torch.float32))
optimizer = optim.Adam(plain_unet.parameters(), lr=1e-4)

def dice_coeff(pred, target, threshold=0.5):
    smooth = 1e-6
    pred = (pred > threshold).float()
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

def train_model(model, train_loader, valid_loader, criterion, optimizer, best_model_path, num_epochs=10, patience=3):
    best_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Training loop
        model.train()
        running_loss = 0.0
        for images, masks in tqdm(train_loader):
            optimizer.zero_grad()  # Clear gradients

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks.unsqueeze(1).float())  # Add channel dimension for masks
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Training loss: {epoch_loss:.4f}")

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, masks in valid_loader:
                outputs = model(images)
                loss = criterion(outputs, masks.unsqueeze(1).float())
                val_loss += loss.item()

        val_loss /= len(valid_loader)
        print(f"Validation loss: {val_loss:.4f}")

        # Early stopping check
        if val_loss < best_loss:
            best_loss = val_loss
            early_stopping_counter = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"Model saved with validation loss: {best_loss:.4f}")
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping")
                break

    print("Training complete")


In [ ]:
best_unet_water_model_path = 'best_unet_water_model_0108.pth'
train_model(plain_unet, train_loader_water, valid_loader_water, criterion, optimizer, best_unet_water_model_path, num_epochs=10, patience=3)

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import os
from torchvision.utils import save_image

import torch
import os
from torchvision.utils import save_image

# Function to calculate true positives, true negatives, false positives, and false negatives
def calculate_tp_fp(pred, mask):
    pred_flat = pred.flatten()
    mask_flat = mask.flatten()
    # True positives are where both pred and mask are 1
    true_positives = np.sum((pred_flat == 1) & (mask_flat == 1))
    true_negatives = np.sum((pred_flat == 0) & (mask_flat == 0))
    # False positives are where pred is 1 but mask is 0
    false_positives = np.sum((pred_flat == 1) & (mask_flat == 0))
    false_negatives = np.sum((pred_flat == 0) & (mask_flat == 1))
    return true_positives, true_negatives, false_positives, false_negatives
# Calculate Dice, IoU, and F-score
def calculate_dice(tp, fp, fn):
    return 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0

def calculate_iou(tp, fp, fn):
    return tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0

def calculate_fscore(precision, recall):
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

def evaluate_model(model, test_loader, save_dir='wildfire-week8/test/testloader-predict', original_images_dir='wildfire-week8/test/testloader-original'):
    model.eval()  # Set the model to evaluation mode
    all_images = []
    all_masks = []
    
    # Create directories to save predicted masks and original images
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(original_images_dir, exist_ok=True)
    
    # Collect all images and masks from the test loader
    for images, masks in test_loader:
        all_images.append(images)
        all_masks.append(masks)
    all_images = torch.cat(all_images, dim=0)
    all_masks = torch.cat(all_masks, dim=0)

    tp_total = tn_total = fp_total = fn_total = 0
    with torch.no_grad():
        for idx in range(len(all_images)):
            image = all_images[idx].unsqueeze(0).to(torch.float32)  # Get the image
            mask = all_masks[idx]  # Get the corresponding mask
            output = model(image)  # Get prediction from the model
            prediction = (output > 0.5).float().squeeze()  # Binarize the output (threshold = 0.5)
            
            # Prepare image, mask, and prediction for saving
            img_np = image.squeeze().permute(1, 2, 0).cpu().numpy()  # (C, H, W) -> (H, W, C)
            mask_np = mask.cpu().numpy()  # Convert mask to numpy
            pred_np = prediction.cpu().numpy()  # Convert prediction to numpy
            
            # Calculate precision, recall, dice, iou, and fscore for the current prediction
            tp, tn, fp, fn = calculate_tp_fp(pred_np, mask_np)
            tp_total += tp
            tn_total += tn
            fp_total += fp
            fn_total += fn
            
            # Save the predicted mask as an image
            pred_image = torch.tensor(pred_np).unsqueeze(0)  # Add channel dimension
            save_image(pred_image, os.path.join(save_dir, f'predicted_mask_{idx}.png'), normalize=True)

            # Save the original image
            save_image(image.squeeze(0), os.path.join(original_images_dir, f'original_image_{idx}.png'), normalize=True)

    # Calculate metrics
    pixel_accuracy = (tp_total + tn_total) / (tp_total + tn_total + fp_total + fn_total)
    precision = tp_total / (tp_total + fp_total) if (tp_total + fp_total) > 0 else 0
    recall = tp_total / (tp_total + fn_total) if (tp_total + fn_total) > 0 else 0
    dice = calculate_dice(tp_total, fp_total, fn_total)
    iou = calculate_iou(tp_total, fp_total, fn_total)
    fscore = calculate_fscore(precision, recall)
    
    print(f"Pixel Accuracy: {pixel_accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, Dice: {dice:.4f}, IoU: {iou:.4f}, F-score: {fscore:.4f}")



In [ ]:
# Example of calling this function
evaluate_model(plain_unet, test_loader_water)

In [ ]:
def save_predictions_with_overlays(model, test_loader, output_folder):
    """
    Displays images with overlayed ground truth and predicted masks, saving only the predicted overlay images.

    Parameters:
    - model: Trained PyTorch model used for predictions.
    - test_loader: DataLoader providing test images and masks.
    - output_folder: Folder where overlayed images will be saved.
    """
    model.eval()  # Set the model to evaluation mode
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists
    
    water_images = []
    water_masks = []
    
    # Collect images with water (mask has value 1)
    for images, masks in test_loader:
        for i in range(images.size(0)):  # Iterate over batch
            # if torch.sum(masks[i] == 1) > 0:  # If the mask contains water (label 1)
            water_images.append(images[i].unsqueeze(0))
            water_masks.append(masks[i].unsqueeze(0))
    
    if len(water_images) == 0:
        print("No water images found in the test set.")
        return
    
    # Convert lists to tensors
    water_images = torch.cat(water_images, dim=0)
    water_masks = torch.cat(water_masks, dim=0)
    
    # Define the new mask color as RGB normalized values
    new_color = np.array([15/255, 94/255, 156/255])  # Normalize RGB values for 0f5e9c
    alpha = 0.9  # Transparency factor

    with torch.no_grad():
        for idx in range(len(water_images)):
            image = water_images[idx].to(torch.float32)  # Get the image
            mask = water_masks[idx].squeeze(0)  # Get the corresponding mask
            
            output = model(image.unsqueeze(0))  # Get prediction from the model
            prediction = (output > 0.5).float().squeeze(0)  # Binarize the output (threshold = 0.5)
            
            # Prepare image, mask, and prediction for overlay
            img_np = image.squeeze().permute(1, 2, 0).cpu().numpy()  # (C, H, W) -> (H, W, C)
            img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min())  # Normalize to [0, 1]
            
            mask_np = mask.cpu().numpy()  # Convert mask to numpy
            pred_np = prediction.cpu().numpy().squeeze()  # Convert prediction to numpy and remove extra dimensions
            
            # Overlay mask onto original image (use custom color)
            img_with_mask = img_np.copy()
            img_with_mask[mask_np == 1] = new_color  # Apply custom color for ground truth mask
            img_with_mask = (alpha * img_with_mask + (1 - alpha) * img_np)  # Blend with original image
            
            img_with_pred = img_np.copy()
            img_with_pred[pred_np == 1] = new_color  # Apply the predicted mask with the custom color
            img_with_pred = (alpha * img_with_pred + (1 - alpha) * img_np)  # Blend with original image

            # Save only the predicted overlay image
            img_filename = f"predicted_overlay_{idx}.png"
            img_path = os.path.join(output_folder, img_filename)

            # Plot all three images
            fig, axs = plt.subplots(1, 3, figsize=(15, 5))
            
            # Display original image
            axs[0].imshow(img_np)
            axs[0].set_title('Original Image')
            axs[0].axis('off')
            
            # Display original image with ground truth mask
            axs[1].imshow(img_with_mask)
            axs[1].set_title('Original Image with Ground Truth Mask')
            axs[1].axis('off')
            
            # Display original image with predicted mask
            axs[2].imshow(img_with_pred)
            axs[2].set_title('Original Image with Predicted Mask')
            axs[2].axis('off')
            
            plt.tight_layout()
            plt.show()  # Show the three images
            
            # Save the predicted overlay image only
            plt.imsave(img_path, img_with_pred)  # Save the predicted overlay image
            plt.close(fig)  # Close the figure to avoid display

In [ ]:
save_predictions_with_overlays(unet_model_water, test_loader_water, output_folder)

## Apply Alive Tree Mask

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

def color_based_segmentation(image):
    """Generates a mask for alive trees based on color segmentation."""
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 40, 20])   # Lowered brightness and saturation to capture darker greens
    upper_green = np.array([100, 255, 180])
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

    return green_mask

def process_images(original_folder, water_mask_folder, water_mask_applied_folder, output_format='png'):
    """Processes all images in the specified folders,
       applies tree detection methods, and visualizes results.
       
    Parameters:
    - output_format: The format to save the output images ('png', 'jpg', 'tiff', etc.)
    """
    
    # Get all original images
    original_images = [f for f in os.listdir(original_folder) if f.lower().endswith(('jpg', 'png', 'jpeg'))]
    
    # Loop through each original image file
    for image_file in original_images:
        # Read the original image
        original_image_path = os.path.join(original_folder, image_file)
        original_image = cv2.imread(original_image_path)
        if original_image is None:
            print(f"Error: Could not read original image from {original_image_path}.")
            continue

        # Read the corresponding water mask (e.g., predicted_mask_0.png)
        water_mask_file = f'predicted_mask_{image_file.split(".")[0].split("_")[-1]}.png'
        water_mask_path = os.path.join(water_mask_folder, water_mask_file)
        water_mask = cv2.imread(water_mask_path, cv2.IMREAD_GRAYSCALE)

        if water_mask is None:
            print(f"Error: Could not read water mask from {water_mask_path}.")
            continue

        # Read the water mask applied image (e.g., predicted_overlay_1.png)
        water_mask_applied_file = f'predicted_overlay_{image_file.split(".")[0].split("_")[-1]}.png'
        water_mask_applied_path = os.path.join(water_mask_applied_folder, water_mask_applied_file)
        water_mask_applied_image = cv2.imread(water_mask_applied_path)

        if water_mask_applied_image is None:
            print(f"Error: Could not read water mask applied image from {water_mask_applied_path}.")
            continue

        # Generate the mask for alive trees
        alive_tree_mask = color_based_segmentation(original_image)

        # Exclude areas identified as water in the water mask
        alive_tree_mask[water_mask > 0] = 0  # Set alive tree mask to 0 where water mask is present

        # Create transparent overlay for alive trees
        alive_tree_overlay = np.zeros_like(original_image, dtype=np.uint8)
        alive_tree_overlay[alive_tree_mask > 0] = [0, 255, 0]  # Green for alive trees

        # Combine overlays with the provided water mask applied image
        alpha_trees = 0.8  # Transparency for tree overlay
        final_overlay_image = cv2.addWeighted(water_mask_applied_image, 1, alive_tree_overlay, alpha_trees, 0)

        # Save the final overlay image
        output_filename = f'final_overlay_{os.path.splitext(image_file)[0]}.{output_format}'
        output_path = os.path.join(water_mask_applied_folder, output_filename)  # Save in the same folder
        cv2.imwrite(output_path, final_overlay_image)

        # Visualize the original image, water mask applied image, and final image with tree overlay
        plt.figure(figsize=(15, 5))

        # Show original image
        plt.subplot(1, 3, 1)
        plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')

        # Show water mask applied image
        plt.subplot(1, 3, 2)
        plt.imshow(cv2.cvtColor(water_mask_applied_image, cv2.COLOR_BGR2RGB))
        plt.title('Image with Water Mask Applied')
        plt.axis('off')

        # Show final image with both overlays
        plt.subplot(1, 3, 3)
        plt.imshow(cv2.cvtColor(final_overlay_image, cv2.COLOR_BGR2RGB))
        plt.title('Final Image with Alive Tree Overlay')
        plt.axis('off')

        plt.show()

    print(f"Final overlay images saved in '{water_mask_applied_folder}' with format '{output_format}'.")



In [ ]:
# Example usage
original_folder = 'wildfire-week8/test/testloader-original'        # Change this to your image folder path
water_mask_folder = 'wildfire-week8/test/testloader-predict'
water_mask_applied_folder = 'wildfire-week8/test/water-overlay'# Change this to your water mask folder path
# output_folder = 'wildfire-week8/test/hsv'  
process_images(original_folder, water_mask_folder, water_mask_applied_folder)

In [ ]:
# import cv2
# import numpy as np
# import os
# import matplotlib.pyplot as plt

# def color_based_segmentation(image):
#     """Generates green and combined (green + yellow) masks for alive trees based on color segmentation."""
#     hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
#     # Green color range
#     lower_green = np.array([35, 50, 40])
#     upper_green = np.array([100, 255, 200])
#     green_mask = cv2.inRange(hsv_image, lower_green, upper_green)
    
#     # # Refined bright yellow color range
#     # lower_yellow = np.array([15, 120, 180])  # Targeting a brighter yellow with high saturation
#     # upper_yellow = np.array([30, 200, 255])   # Allowing for a broader range without high brightness 
#     # yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
#     lower_green2 = np.array([25, 40, 20])   # Lowered brightness and saturation to capture darker greens
#     upper_green2 = np.array([100, 255, 180])
#     green_mask2 = cv2.inRange(hsv_image, lower_green2, upper_green2)
#     # Combined mask (green + yellow)
#     # combined_mask = cv2.bitwise_or(green_mask, yellow_mask)
    
#     return green_mask, green_mask2

# def process_and_visualize_images(image_folder):
#     """Processes each image, generates green and combined masks, and visualizes the results."""
#     # Get all image files
#     image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('jpg', 'jpeg', 'png'))]
    
#     for image_file in image_files:
#         # Read the original image
#         image_path = os.path.join(image_folder, image_file)
#         image = cv2.imread(image_path)
        
#         if image is None:
#             print(f"Error: Could not read image from {image_path}.")
#             continue
        
#         # Generate green and combined masks
#         green_mask, combined_mask = color_based_segmentation(image)
        
#         # Create color overlays for the masks
#         green_overlay = np.zeros_like(image, dtype=np.uint8)
#         green_overlay[green_mask > 0] = [0, 255, 0]  # Green for green mask
        
#         combined_overlay = np.zeros_like(image, dtype=np.uint8)
#         combined_overlay[combined_mask > 0] = [0, 255, 0]  # Green for the combined mask (both green and yellow)
        
#         # Apply the overlays to the original image
#         alpha = 0.9  # Transparency level
#         green_applied = cv2.addWeighted(image, 1, green_overlay, alpha, 0)
#         combined_applied = cv2.addWeighted(image, 1, combined_overlay, alpha, 0)
        
#         # Visualization
#         plt.figure(figsize=(15, 5))
        
#         # Original image
#         plt.subplot(1, 3, 1)
#         plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
#         plt.title('Original Image')
#         plt.axis('off')
        
#         # Green mask applied on original image
#         plt.subplot(1, 3, 2)
#         plt.imshow(cv2.cvtColor(green_applied, cv2.COLOR_BGR2RGB))
#         plt.title('Green Mask Applied')
#         plt.axis('off')
        
#         # Combined mask applied on original image
#         plt.subplot(1, 3, 3)
#         plt.imshow(cv2.cvtColor(combined_applied, cv2.COLOR_BGR2RGB))
#         plt.title('Combined Mask Applied (Green + Yellow)')
#         plt.axis('off')
        
#         plt.show()


# process_and_visualize_images(original_folder)


## Image Segmentation - Three Types of Trees

### Unet + Resnet50

In [ ]:
# Initialize the model
unet_model_tree = Unet(
    encoder_name="resnet50",  # Choose your backbone (e.g., "resnet34", "efficientnet-b3", etc.)
    encoder_weights="imagenet",  # Use pre-trained weights
    in_channels=3,  # Input channels (e.g., 3 for RGB images)
    classes=4  # Number of output classes
)

# Example input
input_tensor = torch.randn(1, 3, 640, 640)  # Batch size of 1, 3 channels (RGB), 640x640 image
output = unet_model_tree(input_tensor)
print(output.shape)

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset

class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.image_filenames = os.listdir(images_dir)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # Load image and corresponding mask
        image_path = os.path.join(self.images_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.masks_dir, self.image_filenames[idx].replace('.jpg', '') + '_mask.png')

        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Debugging print statements for troubleshooting
        if image is None:
            raise ValueError(f"Image not found or unable to load: {image_path}")
        if mask is None:
            raise ValueError(f"Mask not found or unable to load: {mask_path}")

        # Ensure the image has 3 channels
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError(f"Unexpected image shape: {image.shape}")

        # Convert BGR (OpenCV format) to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Convert pixel value 4 to 0 in the mask
        mask[mask == 4] = 0

        

        # Convert image and mask to PyTorch tensors
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  # Change to (C, H, W)
        mask_tensor = torch.tensor(mask, dtype=torch.long)
        # Optionally apply transformations to the image (e.g., augmentations, normalization)
        if self.transform:
            image = self.transform(image)

        # Return the image and mask as tensors
        return image_tensor, mask_tensor


In [ ]:

# Create datasets
train_dataset_tree = SegmentationDataset(train_images_dir, train_masks_dir, transform)
valid_dataset_tree = SegmentationDataset(valid_images_dir, valid_masks_dir, transform)
test_dataset_tree = SegmentationDataset(test_images_dir, test_masks_dir, transform)

# Create data loaders
train_loader_tree = DataLoader(train_dataset_tree, batch_size=4, shuffle=True)
valid_loader_tree = DataLoader(valid_dataset_tree, batch_size=4, shuffle=False)
test_loader_tree = DataLoader(test_dataset_tree, batch_size=4, shuffle=False)

In [ ]:
import torch
from collections import defaultdict

def count_classes_in_dataloader(dataloader):
    """
    Counts the occurrences of each class in the dataset based on the masks provided by the DataLoader.
    
    Args:
        dataloader (torch.utils.data.DataLoader): The PyTorch DataLoader containing images and masks.
        
    Returns:
        class_counts (dict): A dictionary where keys are class labels and values are their respective counts.
    """
    class_counts = defaultdict(int)  # Dictionary to hold class counts

    # Iterate over the DataLoader batches
    for images, masks in dataloader:
        # Flatten the mask and count occurrences of each class label
        unique_labels, counts = torch.unique(masks, return_counts=True)

        # Update the class_counts dictionary with counts from the current batch
        for label, count in zip(unique_labels, counts):
            class_counts[int(label.item())] += int(count.item())

    return dict(class_counts)  # Convert defaultdict to a regular dict before returning


In [ ]:
class_counts = count_classes_in_dataloader(train_loader_tree)
print(class_counts)

In [ ]:
import torch
import numpy as np

def calculate_class_weights(class_counts):
    """
    Calculate class weights based on the class counts from the dataset.
    
    Args:
        class_counts (dict): A dictionary with class IDs or names as keys and pixel counts as values.
    
    Returns:
        torch.Tensor: A tensor of class weights to be used in nn.CrossEntropyLoss.
    """
    # Get the total number of pixels across all classes
    total_pixels = sum(class_counts.values())

    # Calculate the frequency of each class (number of pixels of class / total pixels)
    class_frequencies = {cls: count / total_pixels for cls, count in class_counts.items()}

    # Invert the frequencies to give higher weights to less frequent classes
    class_weights = {cls: 1.0 / freq if freq > 0 else 0.0 for cls, freq in class_frequencies.items()}

    # Normalize weights so that they sum to 1 or use them directly
    weight_values = list(class_weights.values())
    weights_tensor = torch.tensor(weight_values, dtype=torch.float32)

    return weights_tensor



# Calculate weights based on class counts
class_weights = calculate_class_weights(class_counts)
print("Class Weights:", class_weights)




In [ ]:
import copy
# You can now use these weights in nn.CrossEntropyLoss
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(unet_model_tree.parameters(), lr=1e-4)

# Early stopping parameters
patience = 3  # Number of epochs to wait for improvement
best_unet_model_tree_path = "wildfire-week8/best_unet_model_tree.pth"  # Path to save the best model

# Training loop with early stopping, saving, and restoring the best model
def train_model(model, train_loader, valid_loader, criterion, optimizer, best_model_path, num_epochs=10, patience=3):
    best_valid_loss = float('inf')
    early_stopping_counter = 0
    best_model_weights = copy.deepcopy(model.state_dict())  # Copy initial model weights

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, masks in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

        # Validation
        model.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for images, masks in valid_loader:
                outputs = model(images)
                loss = criterion(outputs, masks)
                valid_loss += loss.item()

        valid_loss = valid_loss / len(valid_loader)
        print(f"Validation Loss: {valid_loss:.4f}")

        # Check for improvement in validation loss
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            early_stopping_counter = 0  # Reset counter if validation loss improves
            best_model_weights = copy.deepcopy(model.state_dict())  # Store best weights
            torch.save(best_model_weights, best_model_path)  # Save best model to disk
            print(f"Validation loss improved to {valid_loss:.4f}. Best model saved to {best_model_path}.")
        else:
            early_stopping_counter += 1
            print(f"No improvement in validation loss for {early_stopping_counter} epoch(s).")

        # Early stopping condition
        if early_stopping_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    # Restore the best model weights after early stopping
    model.load_state_dict(torch.load(best_model_path))
    print("Best model weights restored from saved model.")


In [ ]:
# Train the model with early stopping, saving, and restoring best weights
train_model(unet_model_tree, train_loader_tree, valid_loader_tree, criterion, optimizer, best_unet_model_tree_path, num_epochs=10, patience=3)

In [ ]:
import pandas as pd
def evaluate_model(model, data_loader, criterion, class_names):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_pixels = 0
    correct_predictions = 0

    # Initialize counters for each class
    num_classes = len(class_names)
    class_correct = torch.zeros(num_classes)  # Correct predictions per class
    class_total = torch.zeros(num_classes)    # Total ground truth pixels per class
    class_predicted = torch.zeros(num_classes)  # Total predicted pixels per class

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in data_loader:
            preds = model(images)

            # Ensure labels are of shape [batch_size, height, width]
            if labels.dim() == 4:  # If labels have an extra dimension
                labels = labels.squeeze(1)  # Remove the channel dimension

            # Convert labels to Long type (int64)
            labels = labels.long()

            # Calculate loss
            loss = criterion(preds, labels)
            total_loss += loss.item()

            # Get the predicted class for each pixel
            _, predicted = torch.max(preds, 1)  # Shape: [batch_size, height, width]

            # Update overall accuracy (correct_predictions / total_pixels)
            correct_predictions += (predicted == labels).sum().item()
            total_pixels += labels.numel()  # Total number of pixels in the batch

            # Update class-wise correct and total counts
            for class_index in range(num_classes):
                # Correct pixels for each class
                class_correct[class_index] += ((predicted == class_index) & (labels == class_index)).sum().item()
                # Total ground truth pixels for each class
                class_total[class_index] += (labels == class_index).sum().item()
                # Total predicted pixels for each class
                class_predicted[class_index] += (predicted == class_index).sum().item()

    # Calculate average loss and overall pixel accuracy
    average_loss = total_loss / len(data_loader)
    pixel_accuracy = correct_predictions / total_pixels

    # Calculate pixel accuracy for each class, avoiding division by zero
    class_pixel_accuracy = class_correct / class_total
    class_pixel_accuracy[class_total == 0] = 0  # Set accuracy to 0 for classes with no pixels

    # Calculate precision and recall for each class
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)

    for class_index in range(num_classes):
        if class_total[class_index] > 0:  # Avoid division by zero
            # Precision: True Positives / (True Positives + False Positives)
            precision[class_index] = class_correct[class_index] / class_predicted[class_index] if class_predicted[class_index] > 0 else 0
            # Recall: True Positives / (True Positives + False Negatives)
            recall[class_index] = class_correct[class_index] / class_total[class_index]

    # Create a DataFrame to display results, excluding the first class
    results_df = pd.DataFrame({
        'Class Name': class_names[1:],  # Exclude the first class (background)
        'Pixel Accuracy': class_pixel_accuracy[1:].numpy(),  # Exclude the first class
        'Precision': precision[1:],  # Exclude the first class
        'Recall': recall[1:],  # Exclude the first class
    })

    # Round to three decimal places
    results_df[['Pixel Accuracy', 'Precision', 'Recall']] = results_df[['Pixel Accuracy', 'Precision', 'Recall']].round(3)

    return average_loss, pixel_accuracy, results_df

In [ ]:
class_names = ['Background', 'Beetle Trees', 'Dead Tree', 'Debris']  # Adjust class names as needed
average_loss, pixel_accuracy, results_table = evaluate_model(unet_model_tree, test_loader_tree,torch.nn.CrossEntropyLoss() , class_names)
print(results_table)

### Unet + Moblenet v2

In [ ]:
unet_mobilenet_model = Unet(
    encoder_name="mobilenet_v2",  # Choose your backbone (e.g., "resnet34", "efficientnet-b3", etc.)
    encoder_weights="imagenet",  # Use pre-trained weights
    in_channels=3,  # Input channels (e.g., 3 for RGB images)
    classes=4  # Number of output classes
)
# Example input
input_tensor = torch.randn(1, 3, 640, 640)  # Batch size of 1, 3 channels (RGB), 640x640 image
output = unet_model_tree(input_tensor)
print(output.shape)

In [ ]:
# You can now use these weights in nn.CrossEntropyLoss
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(unet_mobilenet_model.parameters(), lr=1e-4)
best_unet_mobilenet_model_path = "wildfire-week7/best_unet_mobilenet_model_tree.pth"

In [ ]:
# Train the model with early stopping, saving, and restoring best weights
train_model(unet_mobilenet_model, train_loader_tree, valid_loader_tree, criterion, optimizer, best_unet_mobilenet_model_path, num_epochs=10, patience=3)

In [ ]:
average_loss, pixel_accuracy, results_table = evaluate_model(unet_mobilenet_model, test_loader_tree,torch.nn.CrossEntropyLoss() , class_names)
print(results_table)